In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# !pip3 install torch
# !pip3 install torchvision

In [0]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import optim

In [0]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                              ])

# **MLP & Pytorch**

In [0]:
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

In [0]:
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In PyTorch we usually process multiple samples at the same time. Usually we stack up each training vector along rows to create $X$ matrix.
\begin{equation}
X = \begin{pmatrix}
- & - & x_1 & - & -\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
. & . & . & . & .\\
- & - & x_{batch-size} & - & -\\
\end{pmatrix}
\end{equation}

torch.nn.Linear class when instantiated creates an object with a randomly initialised $W$ matrix of dimensions(in_class features, out_class features) and another randomly initialised matrix $b$ of dimensions(out_class features, 1). These dimensions are passed as arguments to the constructor during creation of object. When the forward function of the created object is invoked with parameters $X$, it applies linear transformation $XW + b$ and returns the appropriate matrix

In [7]:
# Build a Neural Network
class MLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(784, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 10)
        self.sigmoid = torch.nn.Sigmoid()
        self.softmax = torch.nn.Softmax(dim = 1)
        self.relu = torch.nn.ReLU()
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1.forward(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.softmax(x)

        return x

In [0]:
# Create an object of class MLP
model = MLP()

# Define the loss
criterion = torch.nn.CrossEntropyLoss()

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr = 0.05)

In [0]:
epochs = 100
for e in range(epochs):
    print("Epoch ", (e + 1))
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        logits = model.forward(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        accuracy = 0
        with torch.no_grad():
            for images, labels in testloader:
                logit = model(images)
                ps = logit#torch.exp(logit)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        print("Loss on training set: ", running_loss)
        print("Accuracy on test set: ", end = " ")
        print((accuracy / len(testloader)).data.numpy())

## **Task**
Apply KNN on CIFAR 10 dataset

In [10]:
trainset = datasets.CIFAR10('./data', download=True, train=True, transform=transform)
testset = datasets.CIFAR10('./data', download=True, train=False, transform=transform)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trainset.data.shape[0], shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=True)

In [0]:
# KNN in pytorch

from collections import defaultdict
import operator

K = 5

for traindata, trainlabel in trainloader:
  traindata = traindata.view(traindata.shape[0], -1)
  accuracy = 0
  for testdata, testlabel in testloader:
    testdata = testdata.view(-1)
    testlabel = testlabel.view(-1)
    dist = torch.norm(traindata - testdata, dim=1, p=None)
    _, idx = dist.topk(K, largest=False)
    top_labels = trainlabel[idx]
    map = defaultdict(int)
    for label in top_labels:
      map[label] += 1 
    label = sorted(list(map.items()), key=operator.itemgetter(1), reverse=True)[0][0]
    if label == testlabel[0]: 
      accuracy += 1
  accuracy /= testset.data.shape[0]
print("accuracy: " + str(accuracy))

In [16]:
# KNN in sklearn

import numpy as np
from sklearn.neighbors import KNeighborsClassifier

X_train, y_train = np.array(trainset.data), np.array(trainset.targets)
X_test, y_test = np.array(testset.data), np.array(testset.targets)
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
X_train = X_train[:1000, :]
y_train = y_train[:1000]
X_test = X_test[:1000, :]
y_test = y_test[:1000]

K = 5
neigh = KNeighborsClassifier(n_neighbors=K)
neigh.fit(X_train,y_train)
# neigh.predict(X_test)
neigh.score(X_test,y_test)

0.22